## Import the necessary libraries

In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

## Function find_notebook

In [2]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [3]:
print(find_notebook('integrates_into_database(Matrix Factorization)'))

integrates_into_database(Matrix Factorization).ipynb


## Class NotebookLoader

In [4]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""

    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)

        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [5]:
# Init a notebook loader
notebook_loader = NotebookLoader()

In [6]:
# Import the algorithms module
matrix_fac_rec_sys = notebook_loader.load_module('integrates_into_database(Matrix Factorization)')

importing Jupyter notebook from integrates_into_database(Matrix Factorization).ipynb
iter = 5 , loss = 47.22462132460583 , RMSE train = 1.5428593350436521
iter = 10 , loss = 31.63080267172637 , RMSE train = 1.1442666532837469
iter = 15 , loss = 21.464647027406585 , RMSE train = 0.9992789181617923
iter = 20 , loss = 14.661314281874741 , RMSE train = 0.9636897361972933
iter = 25 , loss = 10.071069265677322 , RMSE train = 0.9562100038399085

User-based MF, RMSE = 0.9877489439744193


In [7]:
# # Get the unique users in database
# # users_id = users_df['id'].unique()
users_id = [1, 2, 3, 4, 5, 672]
# Update recommendation list for each user
for user_id in users_id:
    print(matrix_fac_rec_sys.get_top_five_movie_for_user_id(matrix_fac_rec_sys.rs.pred_for_user(user_id - 1)))
    matrix_fac_rec_sys.users_collection.update_one(
    {"id": int(user_id)},
    {"$set": {"recommendation_list": matrix_fac_rec_sys.get_top_five_movie_for_user_id(matrix_fac_rec_sys.rs.pred_for_user(user_id - 1))}}
    )

AttributeError: 'MF' object has no attribute 'recommend'